In [1]:
from FlagEmbedding import BGEM3FlagModel

In [2]:
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)  # Setting use_fp16 to True speeds up computation with a slight performance degradation

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

/Users/gmkim/anaconda3/envs/torch-gpu/lib/python3.11/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  colbert_state_dict = t

In [3]:

# sentences_1 = ["What is BGE M3?", "Defination of BM25"]
sentence1 = "BGE M3 is an embedding model supporting dense retrieval, lexical matching and multi-vector interaction."
sentence2 = "BM25 is a bag-of-words retrieval function that ranks a set of documents based on the query terms appearing in each document"

In [4]:
embedding1 = model.encode(sentence1,return_dense=True, return_sparse=True, return_colbert_vecs=True, batch_size=12, max_length=8192,)
embedding2 = model.encode(sentence2,return_dense=True, return_sparse=True, return_colbert_vecs=True, batch_size=12, max_length=8192,)

In [19]:
embedding1

{'dense_vecs': array([-8.535e-05, -6.659e-02, -1.844e-04, ...,  4.318e-02, -2.129e-02,
         1.387e-02], dtype=float16),
 'lexical_weights': defaultdict(int,
             {'335': np.float16(0.1411),
              '11679': np.float16(0.2588),
              '276': np.float16(0.172),
              '363': np.float16(0.269),
              '83': np.float16(0.1274),
              '142': np.float16(0.07336),
              '55720': np.float16(0.2141),
              '59725': np.float16(0.167),
              '3299': np.float16(0.255),
              '8060': np.float16(0.1909),
              '214': np.float16(0.08295),
              '168': np.float16(0.1814),
              '184': np.float16(0.1213),
              '456': np.float16(0.05704),
              '97351': np.float16(0.1573),
              '1405': np.float16(0.0633),
              '75675': np.float16(0.1516),
              '21533': np.float16(0.1057),
              '14858': np.float16(0.1509),
              '136': np.float16(0.01573),
   

In [5]:
similarity = embedding1['dense_vecs'] @ embedding2['dense_vecs'].T
print(similarity)

0.556


In [6]:
print(model.convert_id_to_token(embedding1['lexical_weights']))


{'B': np.float16(0.1411), 'GE': np.float16(0.2588), 'M': np.float16(0.172), '3': np.float16(0.269), 'is': np.float16(0.1274), 'an': np.float16(0.07336), 'embe': np.float16(0.2141), 'dding': np.float16(0.167), 'model': np.float16(0.255), 'support': np.float16(0.1909), 'ing': np.float16(0.08295), 'den': np.float16(0.1814), 'se': np.float16(0.1213), 're': np.float16(0.05704), 'trie': np.float16(0.1573), 'val': np.float16(0.0633), 'lex': np.float16(0.1516), 'ical': np.float16(0.1057), 'match': np.float16(0.1509), 'and': np.float16(0.01573), 'multi': np.float16(0.08417), 've': np.float16(0.1454), 'ctor': np.float16(0.1403), 'interaction': np.float16(0.1526)}


In [7]:
lexical_scores = model.compute_lexical_matching_score(embedding1['lexical_weights'], embedding2['lexical_weights'])
print(lexical_scores)


0.05295


In [8]:
print(model.colbert_score(embedding1['colbert_vecs'], embedding2['colbert_vecs']))


tensor(0.5646)


In [9]:

print(model.compute_score([sentence1, sentence2], max_passage_length=128, # a smaller max length leads to a lower latency
                          weights_for_different_modes=[0.49, 0.02, 0.49])) # weights_for_different_modes(w) is used to do weighted sum: w[0]*dense_score + w[1]*sparse_score + w[2]*colbert_score


{'colbert': 0.5646466612815857, 'sparse': 0.05291748046875, 'dense': 0.55615234375, 'sparse+dense': 0.5364176034927368, 'colbert+sparse+dense': 0.5502498149871826}


In [10]:

print(model.compute_score([sentence1, sentence1], max_passage_length=128, # a smaller max length leads to a lower latency
                          weights_for_different_modes=[0.49, 0.02, 0.49])) # weights_for_different_modes(w) is used to do weighted sum: w[0]*dense_score + w[1]*sparse_score + w[2]*colbert_score


{'colbert': 1.0000001192092896, 'sparse': 0.6005859375, 'dense': 1.0, 'sparse+dense': 0.9843367338180542, 'colbert+sparse+dense': 0.9920117855072021}
